In [5]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 38.4 MB/s eta 0:00:0000:01:00:01


In [ ]:
from transformers import pipeline 
from typing import List, Dict
import gradio as gr
import pymupdf as fitz

pipe = pipeline("summarization") #Hugging face function, which uses complex NLP techniques under the hood to summaries the text.

def summarize(user_data: Dict , history: List) -> str:
    """
    Used by the interface, receives user input and file uploads.

    Args:
        user_data (dict): Dictionary containing 'text' and 'files' keys.
        history (list): Chat history (not used in this function).
    Returns:
        str: Summarized text.
    """
    try:
        message = user_data.get("text", "")
        files = user_data.get("files", [])

        if files == []: #No file uploaded, just plain text
            summary = pipe(message)
            output = summary[0]['summary_text']
            return output
        elif files != [] and message == "": #when only file is given
            if files[0] == ".txt": #pasted text is treated as .txt file
                with open(files[0], 'r') as f:
                    file_content = f.read()
                summary = pipe(file_content)
                output = summary[0]['summary_text']
                return output
            elif files[0] == ".pdf": #use PyMuPDF to read pdf files
                pdf_reader = fitz.open(files[0])
                text = ""
                for page in pdf_reader:
                    text += page.get_text("text")
                pdf_reader.close()
                summary = pipe(text)
                output = summary[0]['summary_text']
                return output
            else:
                return "Unsupported file format. Please upload a .txt or .pdf file."
        else:
            return "Please provide either text or file, not both."
    except Exception as e:
        return f"An error occurred: {str(e)}"


demo = gr.ChatInterface(fn=summarize,
                        title= "Text Summarization Bot",
                        multimodal=True
                        )
demo.launch()

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9c29fcaf1acf3ec0e9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Your max_length is set to 142, but your input_length is only 3. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)
